In [33]:
import scipy.io
import scipy
import matplotlib.pyplot as plt
import numpy as np
import os

In [53]:
ids = '01'
kernel_dir = f'/work/project/MEG_GOD/GOD_dataset/sbj{ids}/kernel'
ind_kernel_filename = 'results_MN_MEG_KERNEL_block001.mat'
common_kernel_filename = 'tess_cortex_pial_low.mat'

ch_index_path = '/work/project/MEG_GOD/GOD_dataset/channel_index.mat'
ch_index_path2 = '/work/project/MEG_GOD/GOD_dataset/channel_ricoh_acc1.mat'

# Kernel

In [10]:
ind_kernel_path = os.path.join(kernel_dir, ind_kernel_filename)
common_kernel_path = os.path.join(kernel_dir, common_kernel_filename)
print(ind_kernel_path)
print(common_kernel_path)
ind_kernel = scipy.io.loadmat(ind_kernel_path)
common_kernel = scipy.io.loadmat(common_kernel_path)

/work/project/MEG_GOD/GOD_dataset/sbj01/kernel/results_MN_MEG_KERNEL_block001.mat
/work/project/MEG_GOD/GOD_dataset/sbj01/kernel/tess_cortex_pial_low.mat


In [12]:
print(ind_kernel.keys())
print('\n')
print(common_kernel.keys())

dict_keys(['__header__', '__version__', '__globals__', 'ImagingKernel', 'ImageGridAmp', 'Std', 'Whitener', 'SourceDecompSa', 'SourceDecompVa', 'nComponents', 'Comment', 'Function', 'Time', 'DataFile', 'HeadModelFile', 'HeadModelType', 'ChannelFlag', 'GoodChannel', 'SurfaceFile', 'Atlas', 'GridLoc', 'GridOrient', 'GridAtlas', 'Options', 'ColormapType', 'DisplayUnits', 'ZScore', 'nAvg', 'Leff', 'DataWhitener', 'History'])


dict_keys(['__header__', '__version__', '__globals__', 'Comment', 'Vertices', 'Faces', 'VertConn', 'VertNormals', 'Curvature', 'SulciMap', 'Atlas', 'iAtlas', 'tess2mri_interp', 'Reg', 'History', 'tess2tess_interp'])


In [50]:
ind_mat = ind_kernel['ImagingKernel']
common_mat = common_kernel['tess2tess_interp']['Wmat'][0,0].toarray()# common_kernel['tess2tess_interp']

In [51]:
print('ind_mat, shape', ind_mat.shape)
print('common_mat, shape', common_mat.shape)

ind_mat, shape (15002, 160)
common_mat, shape (15002, 15002)


# Ch Index

In [79]:
cnt = 0
target_roi_indices = []
roi_blocks = [1, 2, 3, 4, 5, 24, 25, 26, 27, 28]
for i in roi_blocks:
    id_ = str(i).zfill(2)
    roi_path = f'/work/project/MEG_GOD/hyang/BData/ROI/hcp44_{id_}.mat'
    roi = scipy.io.loadmat(roi_path)
    # print(roi.keys())
    print(roi['vertex_ds'].shape) # vertex_ds => down sampled
    cnt+= roi['vertex_ds'].shape[0]
    target_roi_indices.append(roi['vertex_ds'])
target_roi_indices = np.concatenate(target_roi_indices, axis=0) - 1 # 0始まりなので
print(target_roi_indices.shape)
print(cnt)

(47, 1)
(85, 1)
(22, 1)
(32, 1)
(31, 1)
(52, 1)
(84, 1)
(28, 1)
(30, 1)
(38, 1)
(449, 1)
449


# create Kernel 

In [80]:
print('common_mat shape: ', common_mat.shape)
print('ind_mat shape: ', ind_mat.shape)
whole_brain_kernel = common_mat @ ind_mat
print('whole_brain_kernel shape: ', whole_brain_kernel.shape)
target_region_kernel = whole_brain_kernel[target_roi_indices[:,0]]
print('target_region_kernel shape: ', target_region_kernel.shape)

common_mat shape:  (15002, 15002)
ind_mat shape:  (15002, 160)
whole_brain_kernel shape:  (15002, 160)
target_region_kernel shape:  (449, 160)


In [83]:
# more effective solution
target_region_kernel2 = common_mat[target_roi_indices[:,0]] @ ind_mat
print(np.all(target_region_kernel == target_region_kernel2))

True


In [84]:
target_region_kernel2

array([[ 1.24827573e+01, -4.99507740e+00, -1.62593974e+00, ...,
         7.88378056e+00,  6.55078261e+00, -2.20422287e-01],
       [ 1.67562931e+00,  5.96575447e+00, -7.16630738e+00, ...,
         1.00485658e+01,  7.33753902e+01, -2.70995233e+01],
       [ 8.07409642e+00,  3.48792221e+00, -2.31603135e+00, ...,
         2.50704393e+01,  2.02620577e+01,  1.04795787e+01],
       ...,
       [ 7.87640771e-01, -1.26336525e+01, -7.65767730e-01, ...,
        -5.35187564e+01,  2.50645984e+01,  7.24292542e+01],
       [ 1.64869634e+01,  1.30704460e+01, -3.77746358e-02, ...,
        -9.71827266e+01,  1.47162953e+01,  8.69700331e+00],
       [ 1.41121592e+00,  9.56242611e+00, -1.91236401e+00, ...,
        -1.98866969e+01,  1.62664471e+01, -9.43809630e+00]])